In [ ]:
from jmetal.lab.experiment import Experiment, Job, generate_summary_from_experiment
from jmetal.operator import PolynomialMutation, SBXCrossover
from jmetal.util.solution import get_non_dominated_solutions, read_solutions
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.algorithm.multiobjective.nsgaii import NSGAII
from jmetal.core.quality_indicator import HyperVolume, EpsilonIndicator

import matplotlib.pyplot as plt
import numpy as np
from modnsgaii import S_NSGAII

from jmetal.problem import ZDT1, ZDT2
from jmetal.problem import DTLZ1, DTLZ2

from surrogate_models.regressor_chain_surrogate import RegressorChainSurrogate
from surrogate_models.multioutput_learning_surrogate import MultiOutputLearnerSurrogate
from surrogate_models.iSOUPTreeRegressor_surrogate import iSOUPTreeRegressorSurrogate
from surrogate_models.LSTM_surrogate import LSTMSurrogate

In [ ]:
def configure_experiment(problems: list, 
                             n_run: int, 
                             n_population_size: int,
                             n_max_evaluations: int,
                             n_offspring_size: int):
    jobs = []

    for run in range(n_run):
        for problem in problems:
            jobs.append(
                Job(
                    algorithm=S_NSGAII(
                        problem=problem,
                        population_size=n_population_size,
                        offspring_population_size=n_offspring_size,
                        mutation=PolynomialMutation(probability=0.95, distribution_index=20),
                        crossover=SBXCrossover(probability=1.0, distribution_index=20),
                        termination_criterion=StoppingByEvaluations(max_evaluations=n_max_evaluations),
                        batch_sample_percentaje = 0.3,
                        surrogate_ml=LSTMSurrogate(dense_layers=64, activation='tanh', optimizer='adam', epochs=20, batch_size=10, verbose=0),

                    ),
                algorithm_tag='NSGAII surrogate 30% batch size alternate',
                problem_tag=problem.name(),
                run=run,
                )
            )
            

    return jobs

In [ ]:
 # Configure the experiments
jobs = configure_experiment(problems=[DTLZ1()], n_run=1, n_population_size=100, n_max_evaluations=50000, n_offspring_size=100)

# Run the study
output_directory = "data"

experiment = Experiment(output_dir=output_directory, jobs=jobs)
experiment.run()


In [ ]:
# Generate summary file
generate_summary_from_experiment(
    input_dir=output_directory,
    reference_fronts="resources/reference_front",
    quality_indicators=[EpsilonIndicator(), HyperVolume([1.0, 1.0, 1.0])],
)